In [1]:

import sys
sys.path.append('/home/techt/Desktop/a4s')


### Step 1: Initial Data Loading

In [2]:
from utils.preprocess import *
# 1. Identify the paths to the soft file and the matrix file
cohort_dir = '/media/techt/DATA/GEO/Lower_Grade_Glioma/GSE74567'
soft_file, matrix_file = geo_get_relevant_filepaths(cohort_dir)

# 2. Read the matrix file to obtain background information and sample characteristics data
background_prefixes = ['!Series_title', '!Series_summary', '!Series_overall_design']
clinical_prefixes = ['!Sample_geo_accession', '!Sample_characteristics_ch1']
background_info, clinical_data = get_background_and_clinical_data(matrix_file, background_prefixes, clinical_prefixes)

# 3. Obtain the sample characteristics dictionary from the clinical dataframe
sample_characteristics_dict = get_unique_values_by_row(clinical_data)

# 4. Explicitly print out all the background information and the sample characteristics dictionary
print("Background Information:")
print(background_info)
print("Sample Characteristics Dictionary:")
print(sample_characteristics_dict)


Background Information:
!Series_title	"Astrocytoma cell lines transduced to either express GFAP isoforms or to knockdown GFAP isoforms"
!Series_summary	"Modulation of the GFAP cytoskeleton in astrocytoma cells alters processes involved in extracellular matrix remodelling and cell-cell signalling – a transcriptome analysis"
!Series_summary	"Astrocytomas grade IV are malignant brain tumours with no effective treatment and a five year survival rate of only 5%. Expression of Glial Fibrillary Acidic Protein (GFAP) is lower in high astrocytoma grade, but the expression of the splice isoform GFAPδ is similar in low and high-grade astrocytomas. Thus the ratio of GFAPδ/α is increased in high-grade astrocytomas. We studied transcriptome changes in astrocytoma cell lines resulting from an induced alteration of GFAP isoform expression. GFAPα or GFAPδ were increased or decreased by recombinant expression or shRNA mediated knockdown of GFAPpan or GFAPα. We find that the most prominent effects are in

### Step 2: Dataset Analysis and Clinical Feature Extraction

In [3]:
is_gene_available = False
trait_row = age_row = gender_row = None  # set to different values when applicable
convert_trait = convert_age = convert_gender = None  # define the functions when applicable

# Checking if gene expression data is available
is_gene_available = True  # Based on the available information, it appears to be gene expression data.

# Variables' keys from sample characteristics dictionary
trait_row = 0  # Given the sample characteristics, it appears 'tumour line' might be the closest match to 'Lower_Grade_Glioma'.
age_row = None  # No explicit mention or close match for 'age' data found.
gender_row = None  # No explicit mention or close match for 'gender' data found.

def convert_trait(value):
    if 'U373' in value:
        return 0
    elif 'U251' in value:
        return 1
    return None

convert_age = None  # As age data is not available

convert_gender = None  # As gender data is not available

# Save cohort information
save_cohort_info('GSE74567', './preprocessed/Lower_Grade_Glioma/cohort_info.json', is_gene_available, trait_row is not None)

# Clinical Feature Extraction (if applicable)
if trait_row is not None:
    selected_clinical_data = geo_select_clinical_features(clinical_data, 'Lower_Grade_Glioma', trait_row, convert_trait, age_row, convert_age, gender_row, convert_gender)
    csv_path = './preprocessed/Lower_Grade_Glioma/trait_data/GSE74567.csv'
    selected_clinical_data.to_csv(csv_path)
    print(preview_df(selected_clinical_data))


{'GSM1923085': [0], 'GSM1923086': [0], 'GSM1923087': [0], 'GSM1923088': [0], 'GSM1923089': [0], 'GSM1923090': [0], 'GSM1923091': [0], 'GSM1923092': [0], 'GSM1923093': [0], 'GSM1923094': [0], 'GSM1923095': [0], 'GSM1923096': [0], 'GSM1923097': [0], 'GSM1923098': [0], 'GSM1923099': [0], 'GSM1923100': [0], 'GSM1923101': [1], 'GSM1923102': [1], 'GSM1923103': [1], 'GSM1923104': [1], 'GSM1923105': [1], 'GSM1923106': [1], 'GSM1923107': [1], 'GSM1923108': [1], 'GSM1923109': [1], 'GSM1923110': [1], 'GSM1923111': [1], 'GSM1923112': [1], 'GSM1923113': [1], 'GSM1923114': [1], 'GSM1923115': [1], 'GSM1923116': [1], 'GSM1923117': [1], 'GSM1923118': [1], 'GSM1923119': [1], 'GSM1923120': [1], 'GSM1923121': [1], 'GSM1923122': [1], 'GSM1923123': [1], 'GSM1923124': [1]}


### Step 3: Gene Data Extraction

In [4]:
# 1. Use the get_genetic_data function from the library to get the gene_data from the matrix_file previously defined.
gene_data = get_genetic_data(matrix_file)

# 2. Print the first 20 row ids for the following step.
print(gene_data.index[:20])


Index(['A_23_P100001', 'A_23_P100022', 'A_23_P100056', 'A_23_P100074',
       'A_23_P100127', 'A_23_P100141', 'A_23_P100189', 'A_23_P100196',
       'A_23_P100203', 'A_23_P100220', 'A_23_P100240', 'A_23_P10025',
       'A_23_P100292', 'A_23_P100315', 'A_23_P100326', 'A_23_P100344',
       'A_23_P100355', 'A_23_P100386', 'A_23_P100392', 'A_23_P100420'],
      dtype='object', name='ID')


### Step 4: Gene Identifier Review

In [5]:
requires_gene_mapping = True


### Step 5: Gene Annotation (Conditional)

In [6]:
# 1. Use the 'get_gene_annotation' function from the library to get gene annotation data from the soft file.
gene_annotation = get_gene_annotation(soft_file)

# 2. Use the 'preview_df' function from the library to preview the data and print out the results.
print("Gene annotation preview:")
print(preview_df(gene_annotation))


Gene annotation preview:
{'ID': ['(+)E1A_r60_1', '(+)E1A_r60_3', '(+)E1A_r60_a104', '(+)E1A_r60_a107', '(+)E1A_r60_a135'], 'SPOT_ID': ['(+)E1A_r60_1', '(+)E1A_r60_3', '(+)E1A_r60_a104', '(+)E1A_r60_a107', '(+)E1A_r60_a135'], 'CONTROL_TYPE': ['pos', 'pos', 'pos', 'pos', 'pos'], 'REFSEQ': [nan, nan, nan, nan, nan], 'GB_ACC': [nan, nan, nan, nan, nan], 'GENE': [nan, nan, nan, nan, nan], 'GENE_SYMBOL': [nan, nan, nan, nan, nan], 'GENE_NAME': [nan, nan, nan, nan, nan], 'UNIGENE_ID': [nan, nan, nan, nan, nan], 'ENSEMBL_ID': [nan, nan, nan, nan, nan], 'TIGR_ID': [nan, nan, nan, nan, nan], 'ACCESSION_STRING': [nan, nan, nan, nan, nan], 'CHROMOSOMAL_LOCATION': [nan, nan, nan, nan, nan], 'CYTOBAND': [nan, nan, nan, nan, nan], 'DESCRIPTION': [nan, nan, nan, nan, nan], 'GO_ID': [nan, nan, nan, nan, nan], 'SEQUENCE': [nan, nan, nan, nan, nan]}


### Step 6: Gene Identifier Mapping

In [7]:
# 1. Determine keys from the annotation dictionary for mapping.
identifier_key = 'ID'
gene_symbol_key = 'GENE_SYMBOL'

# 2. Get the dataframe storing the mapping between probe IDs and genes.
gene_mapping = get_gene_mapping(gene_annotation, identifier_key, gene_symbol_key)

# 3. Apply the mapping.
gene_data = apply_gene_mapping(gene_data, gene_mapping)


### Step 7: Data Normalization and Merging

In [8]:
# 1. Normalize the obtained gene data with the 'normalize_gene_symbols_in_index' function from the library.
normalized_gene_data = normalize_gene_symbols_in_index(gene_data)
gene_csv_path = './preprocessed/Lower_Grade_Glioma/gene_data/GSE74567.csv'
normalized_gene_data.to_csv(gene_csv_path)

# 2. Merge the clinical and genetic data with the 'geo_merge_clinical_genetic_data' function from the library.
merged_data = geo_merge_clinical_genetic_data(selected_clinical_data, normalized_gene_data)

# 3. Determine whether the trait and some demographic attributes in the data is severely biased, and remove biased attributes.
trait_biased, unbiased_merged_data = judge_and_remove_biased_features(merged_data, 'Lower_Grade_Glioma')

# If the trait is not severely biased, save the cohort information and the merged data.

# 4. Save the cohort information.
save_cohort_info('GSE74567', './preprocessed/Lower_Grade_Glioma/cohort_info.json', True, True, trait_biased, unbiased_merged_data)

if not trait_biased:
    # 5. If the trait is not severely biased, save the merged data to a csv file.
    csv_path = './preprocessed/Lower_Grade_Glioma/GSE74567.csv'
    unbiased_merged_data.to_csv(csv_path)


For the feature 'Lower_Grade_Glioma', the least common label is '0.0' with 16 occurrences. This represents 40.00% of the dataset.
The distribution of the feature 'Lower_Grade_Glioma' in this dataset is fine.

